<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px" />

# Excel Madness Lab!

_Author:_ Tim Book

## Our Mission
We work for a large supermarket chain, with stores in 10 major cities that happen to coincide with General Assembly campuses. However, this company's idea of a "database" is just a bunch of Excel spreadsheets! In order to analyze our data, we're going to need to process the existing data into a form we can use. **Our end goal is to have one csv per city.**

## Cleanup Duty!
It is a hard truth that data scientists spend a large majority of their time cleaning data. Data never arrives on our desks in exactly the format in which we want it, and it's up to us to transform it to a workable format.

Being good cleaning, moving, and reshaping data is in itself a valuable and employable job skill. If you follow these directions exactly, we will walk through constructing an automated process for processing data from this supermarket chain.

# Part I: Processing

### Step 1: Imports and the `os` library
We're going to import three libraries: numpy, pandas, and `os`.

In [1]:
# Import libraries here.
import os
import numpy as np
import pandas as pd

The `os` library is extremely useful for performing system commands from within Python. Let's get two pieces of overhead out of the way now:

1. Create an `output` folder using `os.mkdir()`
2. Create a variable called `files` that is the list of files in the `data` folder using `os.listdir()`

**WARNING:** The `os.mkdir()` function will give you an error if you try to make a folder that already exists!

In [2]:
# Create an output folder.
os.mkdir("output")

# Create a files variable that contains all of our data files.
files = os.listdir("data")

In [3]:
files[:5]

['Jan 29.xlsx', 'Jan 3.xlsx', 'Jan 31.xlsx', 'Jan 19.xlsx', 'Jan 11.xlsx']

### Step 2: Process one data frame
It looks like we have data for the month of January. 31 files of 10 sheets each! Luckily they are all in the same format. So let's read just one in and process that.

In [4]:
# Read in data from your city from January 1st.
jan1 = pd.read_excel("data/Jan 1.xlsx", sheet_name="Washington, DC")
jan1.head()

,prodcode,price_eu,weight_kg,quantity
0,4011,1.434955,1.125474,182
1,4404,2.882103,1.407820,363
2,4272,0.547080,4.765729,309
3,4131,2.030937,6.794560,354
4,4240,1.658934,2.978144,347


### Step 2a: Convert to 'Merican columns
For whatever reason, our data are stored in euros and kilograms. Create `price_usd` and `weight_lb` columns. There are 2.2 pounds per kilogram, and 1.1 dollars per euro.

In [5]:
jan1["price_usd"] = 1.1 * jan1["price_eu"]
jan1["weight_lb"] = 2.2 * jan1["weight_kg"]

### Step 2b: Merge in product names
You'll notice we also have a `plu-codes.csv` file containing actual product names matched up against their price lookup (PLU) codes. Let's merge these product names onto our Jan 1 data.
* _Hint 1:_ What kind of merge is this? Right, left, inner, outer, etc.?
* _Hint 2:_ Pay special attention to column names!

In [6]:
plu = pd.read_csv("plu-codes.csv")

In [7]:
jan1 = pd.merge(jan1, plu, how="left", left_on="prodcode", right_on="plu_code")
jan1.head()

,prodcode,price_eu,weight_kg,quantity,price_usd,weight_lb,product,plu_code
0,4011,1.434955,1.125474,182,1.578451,2.476043,Banana,4011
1,4404,2.882103,1.407820,363,3.170313,3.097204,Peach,4404
2,4272,0.547080,4.765729,309,0.601788,10.484605,Grapes,4272
3,4131,2.030937,6.794560,354,2.234031,14.948032,Apple (Fuji),4131
4,4240,1.658934,2.978144,347,1.824828,6.551917,Blueberries,4240


### Step 2c: Drop unnecessary columns
We've created some extraneous columns. Drop the old price and weight columns, as well as any redundant columns.

In [8]:
jan1.drop(["price_eu", "weight_kg", "plu_code"], axis=1, inplace=True)

In [9]:
jan1.head()

,prodcode,quantity,price_usd,weight_lb,product
0,4011,182,1.578451,2.476043,Banana
1,4404,363,3.170313,3.097204,Peach
2,4272,309,0.601788,10.484605,Grapes
3,4131,354,2.234031,14.948032,Apple (Fuji)
4,4240,347,1.824828,6.551917,Blueberries


### Step 2d: Add the date
Simply create a new `date` column that is the date this data was collected. For example, if this is from `Jan 1.xlsx`, this column should be full of `Jan 1`.

In [10]:
jan1["date"] = "Jan 1"

### Step 3: Write a function that conducts all of Step 2
This function should import a **filename and a city name** and return a fully processed DataFrame. That is, the function should:
1. Read in the data from the given file and city.
1. Create USD and pound columns.
1. Merge in product names.
1. Drop unnecessary columns.
1. Add a date column

In [11]:
def process_data(file, city):
    df = pd.read_excel("data/" + file, sheet_name=city)
    df["price_usd"] = 1.1 * df["price_eu"]
    df["weight_lb"] = 2.2 * df["weight_kg"]
    df = pd.merge(df, plu, how="left", left_on="prodcode", right_on="plu_code")
    df.drop(["price_eu", "weight_kg", "plu_code"], axis=1, inplace=True)
    df["date"] = file.replace(".xlsx", "")
    return df

Test your function out on a new file and city!

In [12]:
process_data("Jan 14.xlsx", "Denver").head()

,prodcode,quantity,price_usd,weight_lb,product,date
0,4272,113,1.449998,7.331218,Grapes,Jan 14
1,4131,282,2.136065,7.877898,Apple (Fuji),Jan 14
2,4159,151,3.213403,11.614127,Onion,Jan 14
3,4078,362,2.437546,20.027934,Corn,Jan 14
4,4958,117,1.829635,7.076061,Lemon,Jan 14


### Step 4: Process all of January's data
For each spreadsheet, process the data and store the resulting DataFrame in one big list. **You only need to do this for your city!**

* _Hint 1:_ A listcomp would make this whole step one line of code!
* _Hint 2:_ You've already made that `files` variable to help you here.

In [13]:
jan_list = [process_data(file, "Washington, DC") for file in files]

### Step 5: Concatenate all DataFrames from Step 4 into one large DataFrame
* _Hint:_ Is there a function in `pandas` that can do something like this for us? This is also just one line of code!

In [14]:
jan_full = pd.concat(jan_list, axis=0)

### Step 6: Do this for all cities, write data
Here's the big one. For each city, process and the data as in steps 3-5, and then write the data to our `output` folder. Below is a dictionary of city name to desired output file name.

Before writing your DataFrame, do the following:
* Add a `city` column
* Reorder the columns into the following order:


| city | date | product | prodcode | quantity | weight_lb | price_usd |
|---|---|---|---|---|---|---|

* _Hint:_ You can reorder DataFrame columns simply by writing over your DataFrame with itself, but specifying the specific column order with `.loc`. For example:

`print(df)`

| b | c | a |
|---|---|---|
| 1 | 2 | 3 |

`df = df.loc[:, ["a", "b", "c"]]`

`print(df)`

| a | b | c |
|---|---|---|
| 3 | 1 | 2 |


In [15]:
city_dict = {
    "Atlanta": "atl.csv",
    "Austin": "atx.csv",
    "Boston": "bos.csv",
    "Chicago": "chi.csv",
    "Denver": "den.csv",
    "Los Angeles": "lax.csv",
    "New York": "nyc.csv",
    "San Francisco": "sf.csv",
    "Seattle": "sea.csv",
    "Washington, DC": "dc.csv"
}

In [16]:
# Loop through city_dict and carry out Step 6 here.
# The keys of city_dict can serve as the sheet name.
# The values of city_dict are what you should name the output .csv files.
# If done correctly, this cell could take almost a minute to run!
for city, file_out in city_dict.items():
    data_list = [process_data(file, city) for file in files]
    data_full = pd.concat(data_list, axis=0)
    data_full["city"] = city
    data_full = data_full.loc[:, 
        ["city", "date", "product", "prodcode", "quantity", "weight_lb", "price_usd"]
    ]
    data_full.to_csv("output/" + file_out, index=False)

# Part II: Checking our answers 
In steps very similar to the ones conducted above...
1. Loop through the files we just wrote to `output`, and read them in, collecting them all in one list
1. Combine all of those DataFrames into one large DataFrame
1. For each city, find the mean `quantity`, `weight_lb`, and `price_usd`.

If you've done everything correct, your answer should look exactly like this:

![](imgs/correct-output.png)

In [17]:
files_out = os.listdir("output")
results_list = [pd.read_csv("output/" + file) for file in files_out]
results = pd.concat(results_list, axis=0)

In [18]:
results.groupby("city")["quantity", "weight_lb", "price_usd"].mean()

,quantity,weight_lb,price_usd
city,,,
Atlanta,302.485306,11.929027,1.937980
Austin,301.048298,12.092775,1.937456
Boston,298.806536,12.063057,1.900973
Chicago,301.686186,12.101555,1.930026
Denver,301.012745,12.129850,1.932088
Los Angeles,301.531276,12.167527,1.918331
New York,299.482863,12.090294,1.912662
San Francisco,298.979895,12.237399,1.925830
Seattle,300.333694,11.925486,1.903315


# Part III (BONUS): Get this process production-ready!
_This part of the lab is optional, but very highly recommended, as the skills developed in this part are extremely common in industry._

For this step, we're going to take this whole process and put it into a production-ready Python script. **ABSOLUTELY NONE OF THIS STEP SHOULD TAKE PLACE IN A JUPYTER NOTEBOOK! PRODUCTIONALIZED ETL (_"Extract, Transform, Load"_) CODE DOES NOT TAKE PLACE IN A JUPYTER NOTEBOOK!**

The instructions are simple: As conducted in this lab, read, transform, and export the data in our Excel files into .csv files. The code should be in a `.py` file and executable from the command line. Here are some hints and tips to guide you:

### Hints, tips, and tricks:
* A good place to start is with the code you've already written. In this notebook, you can click `File > Download as > Python (.py)` to export as a `.py` file. Most of this exercise then comes down to you cleaning this file. (There will be a lot to clean).
* Remember `os.mkdir()` will throw an error if the folder you're trying to make already exists. Maybe you should check to see if it already exists. If it already exists, what should you do? (Remember that `.csv` can be overwritten with no problem.) The functions that can help you with this are all in the `os` library.
* Remember to follow all of the Python best practices we've already learned:
    - All import statements should go at the top of your script.
    - Comment your code. Comments shouldn't explain _what_ code does, but _why_ the code does this.
    - Keep your code DRY (don't repeat yourself) as opposed to WET (write everything twice). All constants should be variables that only need to be changed once. All code should be bottled into functions so you only need to fix it once.
* Make sure not to hardcode "Jan" anywhere. The point is that this code can be run throughout the lifetime of this supermarket's project, which is likely months or years. Keep your code so that if you get February data, you only need to change one tiny piece of the script (probably a file path).